## Using Cluster Ensembles to Identify Psychiatric Patient Subgroups
This is the code that belongs to the paper that has been submitted for publication under the title above.

In [ ]:
import pandas as pd
import numpy as np

from hopkins_statistic import hopkins

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

% matplotlib inline

In [ ]:
data_path = ""

## 1. Loading and preprocessing Youth Self Report (YSR) data

##### Load and preprocess YSR data

In [ ]:
ysr = pd.read_csv(os.path.join(data_path, 'ysr_source.csv'), sep=";")

# Parse date
ysr['DATUM'] = pd.to_datetime(ysr['DATUM'], format='%Y-%m-%d')

# The Ysr contains 112 items, item 56h is a free text item and is dismissed
ysr_items = ysr.columns[[c.startswith("v") for c in ysr.columns]][:120]
ysr_items = ysr_items.drop(['v56h_Andere_problemen_schrijf_op'], 1)

# Only retain YSRs with at most five percent missing values (five percent of 112 items = minimum of six)
ysr = ysr[ysr[ysr_items].isnull().sum(1) <= 6]

# Add id
ysr['id_ysr'] = np.arange(len(ysr)) + 1

##### Preprocess items

In [ ]:
# Preprocess each item
for item in ysr_items:
    
    # Map item responses to numeric responses
    ysr[item] = ysr[item].map({'Helemaal niet (voor zover u/je weet)' : 0,
                               'Een beetje of soms' : 1,
                               'Duidelijk of vaak' : 2
                              })
    
    # If value is missing, imputed with median value
    ysr[item] = ysr[item].fillna(ysr[item].median())

##### Compute outcome scales

In [ ]:
# Relevant items for each of the eight outcome scales. (From: http://www.aseba.org/forms/ysrprofile.pdf)
scales_map = {'(1) Anxious depressed'        : [14,29,30,31,32,33,35,45,50,52,71,91,112],
              '(2) Withdrawn depressed'      : [5,42,65,69,75,102,103,111],
              '(3) Somatic complaints'       : [47,51,54,'56a','56b','56c','56d','56e','56f','56g'],
              '(4) Social problems'          : [11,12,25,27,34,36,38,48,62,64,79],
              '(5) Thought problems'         : [9,18,40,46,58,66,70,76,83,84,85,100],
              '(6) Attention problems'       : [1,4,8,10,13,17,41,61,78],
              '(7) Rule breaking behaviour'  : [2,26,28,39,43,63,67,72,81,82,90,96,99,101,105],
              '(8) Aggressive behaviour'     : [3,16,19,20,21,22,23,37,57,68,86,87,89,94,95,97,104]}

# Find correct column name related to item number
def find_column(item_no):
    return [i for i in ysr_items if i.startswith("v{}_".format(item_no))][0]

# Sum scales
for scale in scales_map.keys():
    ysr[scale] = ysr[[find_column(item_no) for item_no in scales_map[scale]]].sum(1)

# List of outcome scales names for subsetting
ysr_outcome_scales = list(scales_map.keys())

##### Finalize dataset

In [ ]:
# Remove all ysr with a total score of 0
ysr = ysr[ysr[ysr_outcome_scales].sum(1) > 0]

# Take first if multiple are present
ysr = ysr.sort_values("DATUM")
ysr = ysr.groupby("Patient_id").first().reset_index()

## 2. Compute hopkins statistic

In [ ]:
no_repeats = 10
h = 0

for i in range(no_repeats):
    h += hopkins(ysr[ysr_outcome_scales])

print("Hopkins statistic = {:.3f}".format(h/no_repeats))

## 3. Cluster ensemble modeling: R

Cluster Ensemble modeling is more easily done in R, using the packages NbClust and Dice. We therefore write the YSR file so that we can open it in R. After R script *cluster_ensemble_modeling.r* is applied to the dataset, it writes the clusters which we load in the next step. Although we could potentially integrate Python and R using interface packages, this is not worth the trouble in this case. 

In [ ]:
# Write to csv
ysr[ysr_outcome_scales].to_csv(os.path.join(data_path,  'ysr.csv'), index=False, sep=";")

##########################
### R SCRIPTS RUN HERE ###
##########################

# The output of the R script is a column with YSR observations as rows, and columns
#    - PCA-1              : x-coordinate of the PCA 
#    - PCA-2              : y-coordinate of the PCA
#    - partition_km       : partioning of the kmeans algorithm
#    - partition_gmm      :                   gaussian mixture model algorithm
#    - partition_ap       :                   affinity propagation algorithm
#    - partition_ensemble :                   cluster ensemble

c_result = pd.read_csv(os.path.join(data_path, 'clusters_result.csv'), sep=";").reset_index()

## 4. Visualize clusters

In [ ]:
# We relabel clusters in order to compare them, based on orientation of center of mass relative to the origin
for column_name in ['partition_km', 'partition_gmm', 'partition_ap', 'partition_ensemble']:
    l = list(c_result.groupby(column_name).apply(lambda x : np.arctan2(np.mean(x['PCA-1']), np.mean(x['PCA-2']))).sort_values(ascending=False).index)
    mapping = {k:v for k,v in zip(l, list(np.arange(3)+1))}
    c_result[column_name] = c_result[column_name].map(mapping)

# Create an additional dataframe which allows integrating the clusters to YSR ids
ysr_clusters = pd.concat([ysr[['DATUM', 'id_ysr', 'Patient_id']], c_result[['partition_ensemble']]], axis=1)        

##### PCA plots

In [ ]:
plt.style.use('seaborn-white')
fig, ax = plt.subplots(1, 4, sharex='col', sharey='row', figsize=(12, 3), dpi=350, squeeze=True)

sns.scatterplot(x='PCA-1', 
                y='PCA-2', 
                data=c_result,
                palette=sns.color_palette('hls', 3),
                style='partition_km',
                hue='partition_km',
                legend=False,
                ax=ax[0])

sns.scatterplot(x='PCA-1', 
                y='PCA-2', 
                data=c_result,
                palette=sns.color_palette('hls', 3),
                style='partition_gmm',
                hue='partition_gmm',
                legend=False,
                ax=ax[1])

sns.scatterplot(x='PCA-1', 
                y='PCA-2', 
                data=c_result,
                palette=sns.color_palette('hls', 3),
                style='partition_ap',
                hue='partition_ap',
                legend=False,
                ax=ax[2])

sns.scatterplot(x='PCA-1', 
                y='PCA-2', 
                data=c_result,
                palette=sns.color_palette('hls', 3),
                style='partition_ensemble',
                hue='partition_ensemble',
                legend=False,
                ax=ax[3])

ax[0].set_title("(a) K-means")
ax[1].set_title("(b) Gaussian Mixture Model")
ax[2].set_title("(c) Affinity Propagation")
ax[3].set_title("(d) Cluster Ensemble")

##### Bar Plot

In [ ]:
# Concatenate YSR outcome scales and Cluster Ensemble labels
ysr_clustered = pd.concat([ysr[ysr_outcome_scales], c_result['partition_ensemble']], axis=1)

# Compute median value for each cluster
ysr_clustered = ysr_clustered.groupby("partition_ensemble").median().transpose().reset_index()

# Reshape to long format for seaborn plot
ysr_clustered = pd.melt(ysr_clustered, id_vars='index')

# Descriptive names for legend
ysr_clustered['partition_ensemble'] = ysr_clustered['partition_ensemble'].map({1 : 'Cluster 1', 
                                                                               2 : 'Cluster 2', 
                                                                               3 : 'Cluster 3'})
# Add figure as bar plot
fig = sns.catplot(x='index', 
                  y='value', 
                  kind='bar',
                  data=ysr_clustered, 
                  hue='partition_ensemble', 
                  palette=sns.color_palette('hls', 3),
                  aspect=2.5,
                  legend=False,
                 )

# Add legend, x and y labels, rotate x 
plt.legend(loc='upper right', fontsize=20)
plt.xlabel("Outcome scale", fontsize=20)
plt.ylabel("Median value", fontsize=20)
plt.xticks(rotation=70, fontsize=18)

fig.savefig('outcome_scales_barplot.png', dpi=350)

## 4. Integrate DSM diagnosis

In Dutch healthcare, DSM diagnosis is registered in a administrative construct called a DBC, which also includes some other relevant variables. 

In [ ]:
# Read file
dbc = pd.read_csv(os.path.join(data_path, 'dbc.csv'), sep=";")

# Parse dates
dbc['DiagnoseDatum'] = pd.to_datetime(dbc['DiagnoseDatum'], format='%Y-%m-%d')
dbc['Startdatum'] = pd.to_datetime(dbc['Startdatum'], format='%Y-%m-%d')
dbc['Einddatum'] = pd.to_datetime(dbc['Einddatum'], format='%Y-%m-%d')

# Add length 
dbc['length'] = (dbc['Einddatum'] - dbc['Startdatum']) / pd.Timedelta('1 day')

# Add id
dbc['id_dbc'] = np.arange(len(dbc)) + 1

# Only filled in date and diagnosis
dbc = dbc[dbc['DiagnoseDatum'].notnull()]
dbc = dbc[dbc['diagnosegroep1'].notnull()]

In [ ]:
# Merge
dbc_merged = dbc.merge(ysr_clusters)

# Select within 12 weeks
dbc_merged['daydiff'] = abs((dbc_merged['DATUM'] - dbc_merged['DiagnoseDatum']) / pd.Timedelta('1 day'))
dbc_diagnose = dbc_merged[dbc_merged['daydiff'] <= (12 * 7)]

# Select first if multiple
dbc_diagnose = dbc_diagnose.sort_values('daydiff')
dbc_diagnose = dbc_diagnose.groupby("id_ysr").first().reset_index()

# Crosstab
pd.crosstab(dbc_diagnose['partition_ensemble'], dbc_diagnose['diagnosegroep1omschrijving']).transpose()

## 5. Integrate clinical notes

In [ ]:
# Notes are already integrated in a clinical data warehouse
notes = pd.read_csv(os.path.join(data_path, 'notes.csv'), sep=";")

# Concatenate if multiple notes exist
notes = notes.groupby("id_ysr")['Text'].apply(lambda x : "\n".join(x)).reset_index()

# Merge cluster labels
ysr_notes = notes.merge(ysr_clusters, how='left', left_on='id_ysr', right_on='id_ysr').reset_index()

In [ ]:
# Vectorize to bag of words
count_vect = CountVectorizer(ngram_range=(1,1), max_features=1000, binary=False)
bag_of_words = count_vect.fit_transform(ysr_notes['Text'])
bow_df = pd.DataFrame(bag_of_words.toarray(), columns=[x for x in count_vect.get_feature_names()])

In [ ]:
outcome_dict = {'column' : bow_df.columns}

# Iterate over clusters numbers
for cluster_id in sorted(ysr_notes['partition_ensemble'].unique()):

    # Outcome is defined as cluster label
    y = (ysr_notes['partition_ensemble'] == cluster_id)
    
    # Save spearman correlations
    rhos = []
    
    # Compute correlations
    for c in bow_df.columns:
        rhos.append(stats.spearmanr(bow_df[c], b=y).correlation)
    
    # Add to outcome dictionary
    outcome_dict[cluster_id] = rhos

In [ ]:
pd.DataFrame(outcome_dict).sort_values(1.0, ascending=False)
pd.DataFrame(outcome_dict).sort_values(2.0, ascending=False)
pd.DataFrame(outcome_dict).sort_values(3.0, ascending=False)

## 6. Clinically relevant variables

##### Show differences among clusters
These variables are again extracted from the DBC construct, which registers such variables. 

In [ ]:
# Merge
dbc_merged = dbc.merge(ysr_clusters)

# Select only relevant 
dbc_merged = dbc_merged[(dbc_merged['DATUM'] >= dbc_merged['Startdatum']) & (dbc_merged['DATUM'] <= dbc_merged['Einddatum'])]

# Select first if multiple
dbc_merged = dbc_merged.sort_values('Einddatum', ascending=False)
dbc_merged = dbc_merged.groupby("id_ysr").first().reset_index()

# Measured outcome variables
outcome_vars = ['zorgvraagzwaarte', 'gafstart_ondergrens', 'gafeind_ondergrens', 'length']
                # zorgvraagzwaarte = burden of disease indicator

# Show differences
dbc_merged.groupby("partition_ensemble")[outcome_vars].mean().transpose()

##### Statistical testing

In [ ]:
print("{:25s} {:25s}".format("Variabele", "P-value"))
print("{:25s} {:25s}".format("=========", "========"))

for v in outcome_vars:
    x1 = dbc_merged[dbc_merged['partition_ensemble'] == 1][v]
    x2 = dbc_merged[dbc_merged['partition_ensemble'] == 2][v]
    x3 = dbc_merged[dbc_merged['partition_ensemble'] == 3][v]
    
    x1 = x1[x1.notnull()]
    x2 = x2[x2.notnull()]
    x3 = x3[x3.notnull()]
    
    s, p = stats.kruskal(x1, x2, x3)
    
    print("{:25s} {:.3f}".format(v, p))